In [20]:
from IPython.display import display

In [1]:
# Step 1: Setup and Installations
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469544 sha256=d80b8d03cf9f2c19aac4d048514a17bcbef5643c85ebecea61bc9c98005d732a
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.

In [1]:
!pip install "numpy<2"

In [2]:
# Step 2: import modules and packages
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


In [3]:
from surprise import Dataset, Reader, SVD
from collections import defaultdict

#Load Models, Tokenizer, and Data

In [4]:
# --- Configuration ---
MODEL_PATH = "/content/drive/MyDrive/goemotions_model"
TOKENIZER_NAME = "distilbert-base-uncased"
SPOTIFY_DATA_PATH = "/content/drive/MyDrive/datasets/spotify/data/data.csv"


In [5]:
# --- Load Emotion Detection Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotion_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
emotion_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
emotion_model.to(device)
emotion_model.eval()
print("✅ Emotion detection model and tokenizer loaded.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

✅ Emotion detection model and tokenizer loaded.


In [6]:
# --- Load Spotify Song Data ---
df = pd.read_csv(SPOTIFY_DATA_PATH)
df = df.dropna(subset=['name', 'artists']).copy()
df['song_id'] = df.index # Create a unique song_id for recommendations
print(f"✅ Spotify dataset loaded with {len(df)} songs.")


✅ Spotify dataset loaded with 170653 songs.


In [7]:
# --- Prepare Data for Content-Based Filtering (KNN) ---
# Select relevant audio features for similarity calculation
audio_features = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                  'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
scaler = StandardScaler()
df_features_scaled = scaler.fit_transform(df[audio_features])


In [8]:
# Train the K-Nearest Neighbors model
knn_model = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
knn_model.fit(df_features_scaled)
print("✅ K-Nearest Neighbors model for content-based filtering is ready.")


✅ K-Nearest Neighbors model for content-based filtering is ready.


# Mood Detection and User Simulation

In [9]:
def detect_emotions_from_text(text):
    """Detects emotions from a text input using the fine-tuned model."""
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    probabilities = torch.sigmoid(logits)
    predictions = (probabilities > 0.4).int().cpu().numpy() # Using a 0.4 threshold
    predicted_indices = predictions[0].nonzero()[0]

    if len(predicted_indices) > 0:
        return [emotion_model.config.id2label[idx] for idx in predicted_indices]
    return ["neutral"] # Default to neutral if no strong emotion is detected

# --- Simulate User Data for Collaborative Filtering ---
# In a real system, this would come from your database of user listening history.
# We'll create a sample of 500 users and 1,000,000 interactions.
print("🔧 Simulating user listening data for collaborative filtering...")
n_users = 500
n_interactions = 1_000_000
user_ids = np.random.randint(0, n_users, n_interactions)
song_ids_interactions = np.random.choice(df['song_id'], n_interactions)
ratings = np.random.randint(1, 6, n_interactions) # e.g., 1-5 star rating or implicit listen count

# Create a ratings DataFrame
ratings_df = pd.DataFrame({
    'userID': user_ids,
    'songID': song_ids_interactions,
    'rating': ratings
})

# --- Train SVD Model for Matrix Factorization ---
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userID', 'songID', 'rating']], reader)
trainset = data.build_full_trainset()
svd_model = SVD()
svd_model.fit(trainset)
print("✅ SVD model for collaborative filtering is ready.")

🔧 Simulating user listening data for collaborative filtering...
✅ SVD model for collaborative filtering is ready.


#Step 4: Building the Recommendation Functions 🧠


In [10]:
def get_content_based_recs_knn(song_id, n=10):
    """Gets recommendations based on audio feature similarity using KNN."""
    song_features = df[df['song_id'] == song_id][audio_features]
    song_features_scaled = scaler.transform(song_features)

    distances, indices = knn_model.kneighbors(song_features_scaled)

    # Exclude the song itself (which is always the first result)
    similar_song_indices = indices.flatten()[1:n+1]
    return df.iloc[similar_song_indices]

In [11]:
def get_collaborative_recs_svd(user_id, n=50):
    """
    Gets top N song recommendations for a user using the SVD model.
    This predicts ratings for all songs the user hasn't seen and returns the best ones.
    """
    # Get a list of all song IDs
    all_song_ids = df['song_id'].unique()

    # Get song IDs that the user has already interacted with
    interacted_songs = ratings_df[ratings_df['userID'] == user_id]['songID'].unique()

    # Predict ratings for songs the user has NOT interacted with
    predictions = []
    for song_id in all_song_ids:
        if song_id not in interacted_songs:
            predictions.append(svd_model.predict(user_id, song_id))

    # Sort predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top N song IDs
    top_n_song_ids = [pred.iid for pred in predictions[:n]]
    return df[df['song_id'].isin(top_n_song_ids)]

In [12]:
def get_contextual_recs(recommendations, context=None):
    """
    Filters recommendations based on context (e.g., time of day).
    This is a simple rule-based example.
    """
    if context is None:
        return recommendations

    if context.get('time_of_day') == 'evening':
        # Prefer calmer songs in the evening
        return recommendations[recommendations['energy'] < 0.6]

    if context.get('activity') == 'workout':
        # Prefer energetic songs for a workout
        return recommendations[recommendations['energy'] > 0.7]

    return recommendations

#Step 5: The Hybrid Recommendation Engine 🎶


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:

def hybrid_recommender(user_text, user_id, context=None, top_n=10):
    """
    The main hybrid recommendation function.

    1. Detects mood from text.
    2. Gets collaborative filtering recommendations for the user.
    3. Filters collaborative recs based on mood-audio feature mapping.
    4. Applies contextual filtering.
    5. Returns the top N recommendations.
    """
    print(f"\n▶️ Generating recommendations for user {user_id}...")
    print(f"User Input: '{user_text}'")

    # 1. Mood Detection
    detected_emotions = detect_emotions_from_text(user_text)
    print(f"🧠 Detected Mood(s): {', '.join(detected_emotions)}")

    # 2. Collaborative Filtering (SVD)
    # Get a base list of recommendations personalized to the user
    # collab_recs = get_collaborative_recs_svd(user_id, n=100) # Get a larger pool first
    # NEW, CORRECTED LINE
    collab_recs = get_collaborative_recs_svd(user_id, n=100).copy()

    # 3. Content-Based Filtering based on Mood
    # Define simple rules to map emotions to audio features
    mood_filters = {
        'joy': {'valence': 0.7, 'energy': 0.7},
        'sadness': {'valence': 0.3, 'energy': 0.4},
        'anger': {'valence': 0.4, 'energy': 0.8},
        'excitement': {'valence': 0.8, 'energy': 0.8},
        'fear': {'valence': 0.3, 'energy': 0.6},
        'calm': {'valence': 0.6, 'energy': 0.3}, # Custom addition
        'neutral': {'valence': 0.5, 'energy': 0.5}
    }

    # Average the feature targets if multiple emotions are detected
    target_features = pd.Series([0.0] * len(audio_features), index=audio_features)
    emotions_found = 0
    for emotion in detected_emotions:
        if emotion in mood_filters:
            emotions_found += 1
            target_features['valence'] += mood_filters[emotion]['valence']
            target_features['energy'] += mood_filters[emotion]['energy']

    if emotions_found > 0:
        target_features /= emotions_found
    else: # Fallback to neutral if detected emotion has no rule
        target_features['valence'] = 0.5
        target_features['energy'] = 0.5

    # Calculate similarity score between each recommended song and the target mood vector
    recs_features = collab_recs[audio_features]
    recs_features_scaled = scaler.transform(recs_features)
    target_features_scaled = scaler.transform(pd.DataFrame([target_features]))

    # Use cosine similarity to find songs closest to the mood profile
    similarity = cosine_similarity(recs_features_scaled, target_features_scaled).flatten()
    collab_recs['mood_match_score'] = similarity

    # Sort by the new mood match score and popularity
    mood_filtered_recs = collab_recs.sort_values(by=['mood_match_score', 'popularity'], ascending=[False, False])

    # 4. Context-Based Filtering
    final_recs = get_contextual_recs(mood_filtered_recs, context)

    return final_recs.head(top_n)



In [27]:
def recommend_song(text, context=None):
    user_id=42

    recommendations = hybrid_recommender(text, user_id, context=context)
    display_cols = ['name', 'artists', 'valence', 'energy', 'mood_match_score']
    recs = recommendations[display_cols]

    display(recs)

In [28]:
energetic_text = "This is a huge achievement, I feel so optimistic and proud of my work!"
recommend_song(energetic_text)


▶️ Generating recommendations for user 42...
User Input: 'This is a huge achievement, I feel so optimistic and proud of my work!'
🧠 Detected Mood(s): admiration


,name,artists,valence,energy,mood_match_score
101488,Desde Hoy,"[""Los Yonic's""]",0.599,0.609,0.820899
148867,Holy Mother - 1999 Remaster,['Eric Clapton'],0.312,0.510,0.768162
18477,Love Me Harder,"['Ariana Grande', 'The Weeknd']",0.240,0.714,0.630445
34029,How Could an Angel Break My Heart,"['Kenny G', 'Toni Braxton']",0.150,0.230,0.584576
135048,She's so High - 2012 Remaster,['Blur'],0.672,0.859,0.581190
55089,"Friend, Please",['Twenty One Pilots'],0.204,0.517,0.565671
100506,No Place Like London,"['Stephen Sondheim', 'Len Cariou', 'Victor Gar...",0.309,0.275,0.554255
23349,"Niye Jao Aaji Shesher (From ""Sahadharminee"")","['Chandra', 'Kamal Dasgupta']",0.309,0.261,0.488744
69651,Así Es la Mujer,['Víctor Manuelle'],0.840,0.868,0.468583
90420,L.I.F.E.G.O.E.S.O.N.,['Noah And The Whale'],0.606,0.745,0.455118


In [29]:
sad_text = "I am feeling so down and depressed for a while."
sad_context = {'time_of_day': 'evening'}

recommend_song(sad_text, sad_context)


▶️ Generating recommendations for user 42...
User Input: 'I am feeling so down and depressed for a while.'
🧠 Detected Mood(s): sadness


,name,artists,valence,energy,mood_match_score
148867,Holy Mother - 1999 Remaster,['Eric Clapton'],0.3120,0.510,0.811588
34029,How Could an Angel Break My Heart,"['Kenny G', 'Toni Braxton']",0.1500,0.230,0.665582
55089,"Friend, Please",['Twenty One Pilots'],0.2040,0.517,0.624440
100506,No Place Like London,"['Stephen Sondheim', 'Len Cariou', 'Victor Gar...",0.3090,0.275,0.600436
23349,"Niye Jao Aaji Shesher (From ""Sahadharminee"")","['Chandra', 'Kamal Dasgupta']",0.3090,0.261,0.550963
165694,Ana Le Habiby,['Fairuz'],0.3220,0.256,0.526201
24518,The Cowman's Prayer,"['Roy Rogers', 'Dale Evans']",0.2680,0.182,0.504299
26892,Moon River - Remastered,['Henry Mancini'],0.0652,0.255,0.458801
152324,God's Will,['Martina McBride'],0.1910,0.332,0.452252
11257,How Much I Feel,['Ambrosia'],0.5180,0.513,0.401130


In [30]:
recommend_song("I am in love with the most beautiful girl I have ever met in my entire life.")


▶️ Generating recommendations for user 42...
User Input: 'I am in love with the most beautiful girl I have ever met in my entire life.'
🧠 Detected Mood(s): love


,name,artists,valence,energy,mood_match_score
101488,Desde Hoy,"[""Los Yonic's""]",0.599,0.609,0.820899
148867,Holy Mother - 1999 Remaster,['Eric Clapton'],0.312,0.510,0.768162
18477,Love Me Harder,"['Ariana Grande', 'The Weeknd']",0.240,0.714,0.630445
34029,How Could an Angel Break My Heart,"['Kenny G', 'Toni Braxton']",0.150,0.230,0.584576
135048,She's so High - 2012 Remaster,['Blur'],0.672,0.859,0.581190
55089,"Friend, Please",['Twenty One Pilots'],0.204,0.517,0.565671
100506,No Place Like London,"['Stephen Sondheim', 'Len Cariou', 'Victor Gar...",0.309,0.275,0.554255
23349,"Niye Jao Aaji Shesher (From ""Sahadharminee"")","['Chandra', 'Kamal Dasgupta']",0.309,0.261,0.488744
69651,Así Es la Mujer,['Víctor Manuelle'],0.840,0.868,0.468583
90420,L.I.F.E.G.O.E.S.O.N.,['Noah And The Whale'],0.606,0.745,0.455118
